# Astropy

"The Astropy Project is a community effort to develop a single core package for Astronomy"

Astropy includes many different tools which may be of use to you including (but not limited to):
* units and constants
* time
* tables
* coordinates
* FITS files and WCS
* models and fitting

In addition Astropy includes the concept of affiliated packages:  "An affiliated package is an astronomy-related Python package that is not part of the astropy core package, but has requested to be included as part of the Astropy project’s community."  Some highlights include:

* ginga FITS viewer
* montage image mosaicing engine
* APLpy (Astronomical Plotting Library in Python)

# Units

In [1]:
from astropy import units as u

In [2]:
distance = 3800*u.km
print(distance)
print(type(distance))
print(distance.value)
print(distance.unit)

3800.0 km
<class 'astropy.units.quantity.Quantity'>
3800.0
km


In [3]:
print(distance.to(u.m))
print(distance.to(u.m).value)

3800000.0 m
3800000.0


In [4]:
velocity = 100*u.m/u.second
print(velocity)

100.0 m / s


In [5]:
print(distance)
print(velocity)
time = distance / velocity
print(time)

3800.0 km
100.0 m / s
38.0 km s / m


In [6]:
time.decompose()

<Quantity 38000.0 s>

In [227]:
# You can format Quantities as you would a float
print(f"{velocity.to(u.km/u.s):.3f}")
print(f"{velocity.to(u.pc/u.Myr):.3f}")

0.100 km / s
0.102 pc / Myr


In [229]:
# Astropy units even has imperial units in the imperial sub-package
print(f"{velocity:.3f}")
print(f"{velocity.to(u.imperial.mile/u.hour):.3f}")

100.000 m / s
223.694 mi / h


### Units has astronomical units such as M_sun and R_earth

In [9]:
distance.to(u.earthRad)

<Quantity 0.5957887145074552 earthRad>

In [10]:
1*u.M_sun

<Quantity 1.0 solMass>

In [11]:
mass = 1e31*u.kg
mass.to(u.M_sun)

<Quantity 5.028978443919801 solMass>

In [232]:
# There are a number of other ... amusing units as well :)
print(f"{velocity.to(u.earthRad/u.fortnight):.3f}")

18.965 earthRad / fortnight


### Equivalencies

`astropy.units` also handles equivalencies, such as that between wavelength and frequency. To use that feature, equivalence objects are passed to the to() conversion method. For instance, a conversion from wavelength to frequency doesn’t normally work because one is a length and the other is 1/time.

In [12]:
(1000 * u.nm).to(u.Hz)

UnitConversionError: 'nm' (length) and 'Hz' (frequency) are not convertible

In [235]:
(1000 * u.nm).to(u.Hz, equivalencies=u.spectral())

<Quantity 299792457999999.94 Hz>

In [237]:
# or perhaps THz is better
(1000 * u.nm).to(u.THz, equivalencies=u.spectral())

<Quantity 299.79245799999995 THz>

`astropy.units` has numerous built in equivalencies and functions including:
* `brightness_temperature(beam_area, disp)`: Defines the conversion between Jy/beam and “brightness temperature”, in Kelvins
* `doppler_optical(rest)`: Return the equivalency pairs for the optical convention for velocity.
* `doppler_radio(rest)`: Return the equivalency pairs for the radio convention for velocity.
* `doppler_relativistic(rest)`: Return the equivalency pairs for the relativistic convention for velocity.
* `pixel_scale(pixscale)`: Convert between pixel distances (in units of pix) and angular units, given a particular pixscale.
* `temperature()`: Convert between Kelvin, Celsius, and Fahrenheit.

# Constants

In [14]:
from astropy import constants as const

In [15]:
const.c

<<class 'astropy.constants.codata2014.CODATA2014'> name='Speed of light in vacuum' value=299792458.0 uncertainty=0.0 unit='m / s' reference='CODATA 2014'>

In [16]:
const.c.cgs

<Quantity 29979245800.0 cm / s>

In [17]:
const.G

<<class 'astropy.constants.codata2014.CODATA2014'> name='Gravitational constant' value=6.67408e-11 uncertainty=3.1e-15 unit='m3 / (kg s2)' reference='CODATA 2014'>

In [18]:
const.M_sun

<<class 'astropy.constants.iau2015.IAU2015'> name='Solar mass' value=1.9884754153381438e+30 uncertainty=9.236140093538353e+25 unit='kg' reference='IAU 2015 Resolution B 3 + CODATA 2014'>

# Time

In [19]:
from astropy.time import Time

In [20]:
t = Time('1999-01-01T00:00:00.123456789', format='isot', scale='utc')
t

<Time object: scale='utc' format='isot' value=1999-01-01T00:00:00.123>

In [21]:
t.jd

2451179.500001429

In [22]:
t.mjd

51179.000001428896

In [23]:
t2 = Time('2017-10-06T20:00:00.123456789', format='isot', scale='utc')

In [24]:
ndays = t2 - t
ndays

<TimeDelta object: scale='tai' format='jd' value=6853.833391203703>

# Coordinates

Astropy is useful and powerful, but that also means it must be complex.  With concepts like time and units one has to be very specific about what system one is working in.  This is also true of `coordinates` where you have to be aware of differing systems (FK5, ICRS, galactic, alt-azimuth, etc.).  Fortunately, astropy usually makes reasonable assumptions.

In [25]:
from astropy.coordinates import SkyCoord

In [26]:
# All of the below are equivalent
c = SkyCoord(10.625, 41.2, frame='icrs', unit='deg')
c = SkyCoord('00h42m30s', '+41d12m00s', frame='icrs')
c = SkyCoord('00h42.5m', '+41d12m')
c = SkyCoord('00 42 30 +41 12 00', unit=(u.hourangle, u.deg))
c = SkyCoord('00:42.5 +41:12', unit=(u.hourangle, u.deg))
c

<SkyCoord (ICRS): (ra, dec) in deg
    ( 10.625,  41.2)>

In [27]:
c.ra

<Longitude 10.625 deg>

In [28]:
c.ra.hour

0.7083333333333335

In [29]:
c.ra.hms

hms_tuple(h=0.0, m=42.0, s=30.000000000000426)

In [30]:
c.dec.degree

41.2

In [31]:
c.dec.radian

0.7190756518216638

In [32]:
c.to_string('hmsdms')

'00h42m30s +41d12m00s'

In [33]:
c.to_string('hmsdms', sep=':')

'00:42:30 +41:12:00'

### Transforming to other systems

In [34]:
c.galactic

<SkyCoord (Galactic): (l, b) in deg
    ( 121.12334339, -21.6403587)>

In [35]:
c.frame

<ICRS Coordinate: (ra, dec) in deg
    ( 10.625,  41.2)>

In [36]:
c.transform_to('fk5')

<SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    ( 10.62501153,  41.20000147)>

In [37]:
from astropy.coordinates import FK5
c.transform_to(FK5(equinox='J1950'))

<SkyCoord (FK5: equinox=J1950.000): (ra, dec) in deg
    ( 9.94123007,  40.92608525)>

In [38]:
c.transform_to(FK5(equinox='J1950')).to_string('hmsdms')

'00h39m45.8952s +40d55m33.9069s'

### Separation

In [39]:
c1 = SkyCoord(ra=10*u.degree, dec=9*u.degree, frame='icrs')
c2 = SkyCoord(ra=11*u.degree, dec=10*u.degree, frame='fk5')
print(c1.separation(c2))  # Differing frames handled correctly  

1d24m16.3209s


### Distance

In [40]:
c1 = SkyCoord(ra=10*u.degree, dec=9*u.degree, distance=10*u.pc, frame='icrs')
c2 = SkyCoord(ra=11*u.degree, dec=10*u.degree, distance=11.5*u.pc, frame='icrs')
print(c1.separation_3d(c2))

1.5228602415117989 pc


### EarthLocation

In [41]:
from astropy.coordinates import EarthLocation

In [42]:
subaru = EarthLocation.of_site('Subaru')
subaru

<EarthLocation (-5464468.1097167, -2493053.65044845,  2150943.60508102) m>

In [43]:
from astropy.coordinates import AltAz
utcoffset = -10*u.hour  # HST
obstime = Time.now() - utcoffset
target = SkyCoord.from_name('M31')
target_altaz = target.transform_to(AltAz(obstime=obstime,location=subaru))
target_altaz

<SkyCoord (AltAz: obstime=2017-10-03 05:24:24.015901, location=(-5464468.109716696, -2493053.6504484517, 2150943.6050810195) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (az, alt) in deg
    ( 52.65965759,  25.35436877)>

In [44]:
target_altaz.alt, target_altaz.az

(<Latitude 25.354368766817593 deg>, <Longitude 52.659657592547 deg>)

In [238]:
# get barycentric corrections to radial velocity
target.radial_velocity_correction(obstime=obstime, location=subaru).to('km/s')  

<Quantity 7.5509889935291445 km / s>

# Exercises

## Exercise 1) 